In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForMaskedLM

from tqdm import tqdm

import tensorflow as tf
import os
print(f"Tensorflow version: {tf.__version__}")

c:\mc3\envs\nmrv2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tensorflow version: 2.6.0


In [2]:
# Restrict TensorFlow to only allocate 4GBs of memory on the first GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    #tf.config.experimental.set_memory_growth(gpus[0], True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(f"The system contains '{len(gpus)}' Physical GPUs and '{len(logical_gpus)}' Logical GPUs")
  except RuntimeError as e:
    print(e)
else:
    print(f"Your system does not contain a GPU that could be used by Tensorflow!")

The system contains '1' Physical GPUs and '1' Logical GPUs


In [3]:
data = pd.read_json('../../data/data.json')

data = data.filter(['title', 'comment_category'])
data.head()

,title,comment_category
0,"WHO na Kitajsko pošilja strokovnjake, ki bodo ...",2
1,Podgoršek: Prehranska varnost v Sloveniji tren...,3
2,"""Moramo se rešiti"": na desettisoče ljudi prote...",2
3,Andrijanič za mobilno aplikacijo z vsemi stori...,4
4,Ursula von der Leyen zaradi madžarskega zakona...,4


In [4]:
X = (np.array(data['title']))
y = (np.array(data['comment_category']))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
print("Train dataset shape: {0}, \nTest dataset shape: {1} \nValidation dataset shape: {2}".format(X_train.shape, X_test.shape, X_val.shape))

Train dataset shape: (7318,), 
Test dataset shape: (915,) 
Validation dataset shape: (915,)


In [5]:
tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/sloberta")
model = AutoModelForMaskedLM.from_pretrained("EMBEDDIA/sloberta")

In [6]:
def get_token_ids(texts):
    return tokenizer.batch_encode_plus(texts, add_special_tokens=True, padding = True)["input_ids"]

train_token_ids = get_token_ids(list(X_train))
test_token_ids = get_token_ids(list(X_test))

In [7]:
train_data = tf.data.Dataset.from_tensor_slices((tf.constant(train_token_ids), tf.constant(y_train))).batch(12)
test_data = tf.data.Dataset.from_tensor_slices((tf.constant(test_token_ids), tf.constant(y_test))).batch(12)

In [10]:
from transformers import TFCamembertForMaskedLM, TFBertMainLayer
from tensorflow.keras import layers

import tensorflow as tf
class SloBertEmbeddingModel(TFCamembertForMaskedLM):
    def __init__(self, config,
                 cnn_filters=50,
                 dnn_units=512,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model",
                 *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert", trainable = False)
        
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.last_dense = layers.Dense(units=1, activation="linear")

    def call(self, inputs, training = False, **kwargs):        
        bert_outputs = self.bert(inputs, training = training, **kwargs)
        
        l_1 = self.cnn_layer1(bert_outputs[0]) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(bert_outputs[0]) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(bert_outputs[0])
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [11]:
CNN_FILTERS = 100
DNN_UNITS = 256
DROPOUT_RATE = 0.2
NB_EPOCHS = 5

text_model = SloBertEmbeddingModel.from_pretrained('EMBEDDIA/sloberta',
                        from_pt=True,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        dropout_rate=DROPOUT_RATE)

text_model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model SloBertEmbeddingModel: ['roberta.encoder.layer.7.attention.self.key.bias', 'roberta.encoder.layer.6.attention.self.value.bias', 'roberta.encoder.layer.2.attention.self.key.bias', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.encoder.layer.2.attention.self.query.weight', 'roberta.encoder.layer.11.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.1.intermediate.dense.bias', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.10.output.LayerNorm.bias', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.10.attention.self.value.weight', 'roberta.encoder.layer.4.attention.self.query.bias', 'roberta.encoder.layer.4.attention.self.query.weight', 'roberta.encoder.layer.5.attention.self.value.bias', 'roberta.encoder.layer.11.attention

In [12]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
610/610 [==============================] - 49s 52ms/step - loss: 2.5472 - mae: 1.3101
Epoch 2/5
610/610 [==============================] - 35s 57ms/step - loss: 1.9821 - mae: 1.2049
Epoch 3/5
610/610 [==============================] - 36s 58ms/step - loss: 1.9237 - mae: 1.1858
Epoch 4/5
610/610 [==============================] - 36s 59ms/step - loss: 1.8448 - mae: 1.1551
Epoch 5/5
610/610 [==============================] - 37s 60ms/step - loss: 1.7689 - mae: 1.1287


In [13]:
results_predicted = [round(x[0]) for x in text_model.predict(test_data)]
results_true = list(y_test)

print(f"Predicted: {results_predicted[:20]}")
print(f"Truth    : {results_true[:20]}")

Predicted: [3, 2, 2, 1, 2, 2, 1, 2, 2, 1, 3, 2, 2, 2, 2, 2, 3, 3, 2, 2]
Truth    : [4, 3, 2, 2, 1, 1, 0, 0, 2, 0, 4, 4, 2, 3, 1, 3, 3, 3, 2, 0]


In [14]:
from sklearn.metrics import mean_absolute_error

print(f"MAE score: {mean_absolute_error(results_true, results_predicted)}")

MAE score: 1.1398907103825138
